In [2]:
import os
from dotenv import load_dotenv
from typing import TypedDict
from langchain.agents.middleware import dynamic_prompt, ModelRequest
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

load_dotenv()

model = ChatOpenAI(
    model="qwen3-max",
    temperature=0.1,
    max_tokens=1000,
    timeout=30,
    api_key=os.getenv('API_KEY'),
    base_url=os.getenv('BASE_URL')
)

class CustomContext(TypedDict):
    user_name: str

def get_weather(city: str) -> str:
    """Get the weather in a city."""
    return f"The weather in {city} is always sunny!"

@dynamic_prompt
def dynamic_system_prompt(request: ModelRequest) -> str:
    user_name = request.runtime.context["user_name"]
    system_prompt = f"You are a helpful assistant. Address the user as {user_name}."
    return system_prompt

agent = create_agent(
    model,
    tools = [get_weather],
    middleware=[dynamic_system_prompt],
    context_schema=CustomContext,
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    context=CustomContext(user_name="John Smith")
)

for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================

What is the weather in SF?
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_7a732ab9eaf046b88bc8a9ca)
 Call ID: call_7a732ab9eaf046b88bc8a9ca
  Args:
    city: SF
================================= Tool Message =================================
Name: get_weather

The weather in SF is always sunny!
================================== Ai Message ==================================

John Smith, the weather in SF is always sunny!


In [8]:
from langchain.messages import RemoveMessage
from langgraph.graph.message import REMOVE_ALL_MESSAGES
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents import AgentState
from langchain.agents.middleware import before_model
from langchain_core.runnables import RunnableConfig
from langgraph.runtime import Runtime
from typing import Any

@before_model
def trim_messages(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Keep only the last few messages to fit context window."""
    messages = state["messages"]

    if len(messages) <= 3:
        return None

    first_msg = messages[0]
    recent_messages = messages[-3:] if len(messages) % 2 == 0 else messages[-4:]
    new_messages = [first_msg] + recent_messages

    return {
        "messages": [
            RemoveMessage(id=REMOVE_ALL_MESSAGES),
            *new_messages
        ]
    }

agent = create_agent(
    model,
    tools=[],
    middleware=[trim_messages],
    checkpointer=InMemorySaver()
)

config: RunnableConfig = {"configurable": {"thread_id": "1"}}

agent.invoke({"messages": "hi, my name is bob"}, config)
agent.invoke({"messages": "write a short poem about cats"}, config)
agent.invoke({"messages": "now do the same but for dogs"}, config)
final_response = agent.invoke({"messages": "what's my name?"}, config)

final_response["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob! 😊


### After Model

In [14]:
from langchain.agents import AgentState
from langchain.agents.middleware import after_model

@after_model
def validate_response(state: AgentState, runtime: Runtime) -> dict | None:
    """Remove messages containing sensitive words."""
    STOP_WORDS = ["password", "secret"]
    last_message = state["messages"][-1]
    if any(word in last_message.content for word in STOP_WORDS):
        return {"messages": [RemoveMessage(id=last_message.id)]}
    return None

agent = create_agent(
    model,
    tools=[],
    middleware=[validate_response],
    checkpointer=InMemorySaver(),
)

config: RunnableConfig = {"configurable": {"thread_id": "2"}}
agent.invoke({"messages": "My password is 123"}, config)
result = agent.invoke({"messages": "What's my password?"}, config)
result

{'messages': [HumanMessage(content='My password is 123', additional_kwargs={}, response_metadata={}, id='4d4695b8-1449-4d18-ba17-eae446e44fa7'),
  AIMessage(content='I notice you\'ve shared a password ("123"), which is extremely weak and insecure. Here\'s what you should do immediately:\n\n### 🔒 Critical Actions:\n1. **Change this password everywhere it\'s used** (email, social media, banking, etc.)\n2. **Never reuse passwords** across different accounts\n3. **Enable two-factor authentication (2FA)** on all important accounts\n\n### 🛡️ How to Create a Strong Password:\n- **Length**: At least 12 characters\n- **Complexity**: Mix uppercase, lowercase, numbers, and symbols\n- **Unpredictability**: Avoid dictionary words, sequences (123), or personal info\n- **Example**: `7m$Q!vP2#kL9@wR` (but better yet—use a password manager to generate/store)\n\n### 💡 Better Solution: Use a Password Manager\nTools like **Bitwarden** (free), **1Password**, or **KeePass** can:\n- Generate uncrackable pass

================================ Human Message =================================

What is the weather in SF?
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_ef6b426b4ddc4896b17090b2)
 Call ID: call_ef6b426b4ddc4896b17090b2
  Args:
    city: SF
================================= Tool Message =================================
Name: get_weather

It's always sunny in SF!
================================== Ai Message ==================================

John Smith, it's always sunny in SF!
